In [1]:
import pandas as pd
import os
def user_number_to_sort(filename):
    """
    Extracts the numeric part from filenames like 'U123.csv' for sorting.
    Returns 0 if the format is unexpected.
    """
    if filename.endswith('.csv') and filename.startswith('U'):
        try:
            return int(filename[1:-4])  # Remove 'U' and '.csv'
        except ValueError:
            return 0
    return 0


In [ ]:
# main folder path here
filepath = 'C:/Users/dmarc/Desktop/EWELD_Project/EWELD/Electricity Consumption/'  
# list of sections (folders) inside the main directory
sections = [d for d in os.listdir(filepath) if os.path.isdir(os.path.join(filepath, d))]
print(sections) # file names

['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'S']


In [ ]:
dataframes = []

for section in sections:
    section_path = os.path.join(filepath, section)
    divisions = [d for d in os.listdir(section_path) if os.path.isdir(os.path.join(section_path, d))]
    for division in divisions:
        division_path = os.path.join(section_path, division)
        # Get all CSV files and sort them using the helper function
        csv_files = [f for f in os.listdir(division_path) if f.endswith('.csv')]
        csv_files = sorted(csv_files, key=user_number_to_sort)
        for csv_file in csv_files:
            file_path = os.path.join(division_path, csv_file)
            try:
                df = pd.read_csv(file_path)
                df['User'] = os.path.splitext(csv_file)[0]   # e.g., 'U1'
                df['Division'] = division                    # e.g., 'Division_01'
                df['Section'] = section                      # e.g., 'Section_A'
                dataframes.append(df)
            except Exception as e:
                print(f"Error reading {file_path}: {e}")

# Combine all into one DataFrame
if dataframes:
    combined_df = pd.concat(dataframes, ignore_index=True)
    print(combined_df.head())
else:
    print("No dataframes were loaded")

                  Time    Value User  \
0  2016-07-12 10:30:00  20.3325   U1   
1  2016-07-12 10:45:00  16.8611   U1   
2  2016-07-12 11:00:00  16.2412   U1   
3  2016-07-12 11:15:00  18.8448   U1   
4  2016-07-12 11:30:00  17.3570   U1   

                                            Division Section  
0  A01 Crop and animal production, hunting and re...       A  
1  A01 Crop and animal production, hunting and re...       A  
2  A01 Crop and animal production, hunting and re...       A  
3  A01 Crop and animal production, hunting and re...       A  
4  A01 Crop and animal production, hunting and re...       A  


In [3]:
import pandas as pd
import os

# Define directory and file name UserLocation
directory = r'C:/Users/dmarc/Desktop/EWELD_Project/EWELD/User Location/'
file_name = 'U_CT1.csv'
file_path = os.path.join(directory, file_name)
# Remove the .csv extension
location_name = os.path.splitext(file_name)[0]
# Load the DataFrame from the pickle file
df_CT1 = pd.read_csv(file_path)
# Remove the .csv extension and prefix U_
location_name = os.path.splitext(file_name)[0]
location_name = location_name.removeprefix("U_")
# Add the new column
df_CT1['City'] = location_name

In [4]:
print(df_CT1 .head())

  User No. City
0      U14  CT1
1      U19  CT1
2      U50  CT1
3     U103  CT1
4     U125  CT1


In [5]:
'combined_df' in locals() 

True

In [6]:
combined_df.head()

,Time,Value,User,Division,Section
0,2016-07-12 10:30:00,20.3325,U1,"A01 Crop and animal production, hunting and re...",A
1,2016-07-12 10:45:00,16.8611,U1,"A01 Crop and animal production, hunting and re...",A
2,2016-07-12 11:00:00,16.2412,U1,"A01 Crop and animal production, hunting and re...",A
3,2016-07-12 11:15:00,18.8448,U1,"A01 Crop and animal production, hunting and re...",A
4,2016-07-12 11:30:00,17.3570,U1,"A01 Crop and animal production, hunting and re...",A


In [7]:
merged_df = combined_df.merge(
    df_CT1,
    how='left',
    left_on='User',
    right_on='User No.'
)
merged_df['City'] = merged_df['City'].fillna('N/A')
merged_df = merged_df.drop(columns=['User No.'])
print(merged_df .head())

                  Time    Value User  \
0  2016-07-12 10:30:00  20.3325   U1   
1  2016-07-12 10:45:00  16.8611   U1   
2  2016-07-12 11:00:00  16.2412   U1   
3  2016-07-12 11:15:00  18.8448   U1   
4  2016-07-12 11:30:00  17.3570   U1   

                                            Division Section City  
0  A01 Crop and animal production, hunting and re...       A  N/A  
1  A01 Crop and animal production, hunting and re...       A  N/A  
2  A01 Crop and animal production, hunting and re...       A  N/A  
3  A01 Crop and animal production, hunting and re...       A  N/A  
4  A01 Crop and animal production, hunting and re...       A  N/A  


In [22]:
filtered_df = merged_df[merged_df['City'] != 'N/A']


In [23]:
filtered_df.head()

,Time,Value,User,Division,Section,City
2009903,2020-02-29 00:15:00,102.1193,U14,C10 Manufacture of food products,C,CT1
2009904,2020-02-29 00:30:00,105.2034,U14,C10 Manufacture of food products,C,CT1
2009905,2020-02-29 00:45:00,105.8888,U14,C10 Manufacture of food products,C,CT1
2009906,2020-02-29 01:00:00,104.5180,U14,C10 Manufacture of food products,C,CT1
2009907,2020-02-29 01:15:00,104.5180,U14,C10 Manufacture of food products,C,CT1


In [25]:
# Keep only the Division column
filtered_d_s_df = filtered_df.copy()

# Extract Division Name (after the first space)
filtered_d_s_df['Division Name'] = filtered_d_s_df['Division'].apply(lambda x: x.split(' ', 1)[1])

# Extract Div_Sec (first 3 characters)
filtered_d_s_df['Div_Sec'] = filtered_d_s_df['Division'].str[:3]

# Drop the original Division column
filtered_d_s_df = filtered_d_s_df.drop(columns=['Division'])

# Result
print(filtered_d_s_df)

                         Time     Value  User Section City  \
2009903   2020-02-29 00:15:00  102.1193   U14       C  CT1   
2009904   2020-02-29 00:30:00  105.2034   U14       C  CT1   
2009905   2020-02-29 00:45:00  105.8888   U14       C  CT1   
2009906   2020-02-29 01:00:00  104.5180   U14       C  CT1   
2009907   2020-02-29 01:15:00  104.5180   U14       C  CT1   
...                       ...       ...   ...     ...  ...   
54604122  2021-12-31 22:45:00    0.1900  U382       Q  CT1   
54604123  2021-12-31 23:00:00    0.4751  U382       Q  CT1   
54604124  2021-12-31 23:15:00    0.1900  U382       Q  CT1   
54604125  2021-12-31 23:30:00    0.7601  U382       Q  CT1   
54604126  2021-12-31 23:45:00    0.2850  U382       Q  CT1   

                         Division Name Div_Sec  
2009903   Manufacture of food products     C10  
2009904   Manufacture of food products     C10  
2009905   Manufacture of food products     C10  
2009906   Manufacture of food products     C10  
2009907   

In [27]:
unique_City = filtered_d_s_df['User'].unique()
print(unique_City)

['U14' 'U19' 'U50' 'U103' 'U125' 'U137' 'U228' 'U266' 'U268' 'U354' 'U376'
 'U382']


In [ ]:
# Prepare final df
selected_columns = [
    'Time',
    'Value',
    'User',
    'Div_Sec',
    'City'
]
df_ec_CT1 = filtered_d_s_df[selected_columns].copy()
df_ec_CT1.head()


,Time,Value,User,Div_Sec,City
2009903,2020-02-29 00:15:00,102.1193,U14,C10,CT1
2009904,2020-02-29 00:30:00,105.2034,U14,C10,CT1
2009905,2020-02-29 00:45:00,105.8888,U14,C10,CT1
2009906,2020-02-29 01:00:00,104.5180,U14,C10,CT1
2009907,2020-02-29 01:15:00,104.5180,U14,C10,CT1


In [ ]:
df_ec_CT1.tail()

6489105

In [39]:
shape = df_ec_CT1.shape
print("Shape = {}".format(shape))

Shape = (1297821, 5)


In [30]:
df_ec_CT1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1297821 entries, 2009903 to 54604126
Data columns (total 5 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   Time     1297821 non-null  object 
 1   Value    1297821 non-null  float64
 2   User     1297821 non-null  object 
 3   Div_Sec  1297821 non-null  object 
 4   City     1297821 non-null  object 
dtypes: float64(1), object(4)
memory usage: 59.4+ MB


In [32]:
pivot_df_ec_CT1 = df_ec_CT1.pivot_table(
    index=['Time', 'City'],
    columns='User',
    values='Value'
)
pivot_df_ec_CT1 = pivot_df_ec_CT1.reset_index()
pivot_df_ec_CT1

User,Time,City,U103,U125,U137,U14,U19,U228,U266,U268,U354,U376,U382,U50
0,2017-03-18 00:15:00,CT1,NaN,NaN,NaN,NaN,NaN,404.0065,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-03-18 00:30:00,CT1,NaN,NaN,NaN,NaN,NaN,402.3807,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-03-18 00:45:00,CT1,NaN,NaN,NaN,NaN,NaN,413.3547,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-03-18 01:00:00,CT1,NaN,NaN,NaN,NaN,NaN,396.6905,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-03-18 01:15:00,CT1,NaN,NaN,NaN,NaN,NaN,407.6645,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189114,2022-08-08 22:45:00,CT1,NaN,NaN,2.6072,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189115,2022-08-08 23:00:00,CT1,NaN,NaN,2.3175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189116,2022-08-08 23:15:00,CT1,NaN,NaN,3.3315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189117,2022-08-08 23:30:00,CT1,NaN,NaN,3.6212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
# Calculate MIN and MAX time (corrected from your original code)
min_time = pivot_df_ec_CT1['Time'].min()  # Earliest time
max_time = pivot_df_ec_CT1['Time'].max()  # Latest time

# Create a DataFrame to show the range
time_input_range_df = pd.DataFrame({
    'Min Time': [min_time],
    'Max Time': [max_time]
})

print(time_input_range_df)

             Min Time            Max Time
0 2017-03-18 00:15:00 2022-08-08 23:45:00


In [35]:
pivot_df_ec_CT1 .info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189119 entries, 0 to 189118
Data columns (total 14 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    189119 non-null  object 
 1   City    189119 non-null  object 
 2   U103    81173 non-null   float64
 3   U125    150242 non-null  float64
 4   U137    180183 non-null  float64
 5   U14     85439 non-null   float64
 6   U19     152203 non-null  float64
 7   U228    161532 non-null  float64
 8   U266    64184 non-null   float64
 9   U268    85112 non-null   float64
 10  U354    85439 non-null   float64
 11  U376    57436 non-null   float64
 12  U382    64511 non-null   float64
 13  U50     130367 non-null  float64
dtypes: float64(12), object(2)
memory usage: 20.2+ MB


In [44]:
shape = pivot_df_ec_CT1.shape
print("Data Frane pivot_df_ec_CT1 shape = {}".format(shape))

print('----\n')
missing_values = pivot_df_ec_CT1.drop(columns=['Time', 'City']).isna().sum()
missing_values_dict = missing_values.to_frame
print("missing_values by User {}".format(missing_values_dict))


Data Frane pivot_df_ec_CT1 shape = (189119, 14)
----

missing_values by User <bound method Series.to_frame of User
U103    107946
U125     38877
U137      8936
U14     103680
U19      36916
U228     27587
U266    124935
U268    104007
U354    103680
U376    131683
U382    124608
U50      58752
dtype: int64>


create a DataFrame showing the minimum and maximum time for each user 

In [ ]:
#Ensure Time is a datetime column
pivot_df_ec_CT1['Time'] = pd.to_datetime(pivot_df_ec_CT1['Time'])


In [ ]:
# Get user columns (exclude Time and City)
user_columns = pivot_df_ec_CT1.columns.difference(['Time', 'City'])

# Initialize dictionaries to store results
min_times = {}
max_times = {}

for user in user_columns:
    # Filter rows where the user's value is not NaN
    valid_rows = pivot_df_ec_CT1[pivot_df_ec_CT1[user].notna()]
    
    if not valid_rows.empty:
        min_times[user] = valid_rows['Time'].min()
        max_times[user] = valid_rows['Time'].max()
    else:
        min_times[user] = pd.NaT  # Handle users with no data
        max_times[user] = pd.NaT

# Create the summary DataFrame
time_range_df = pd.DataFrame({
    'User': list(min_times.keys()),
    'Min Time': list(min_times.values()),
    'Max Time': list(max_times.values())
})


In [47]:
time_range_df

,User,Min Time,Max Time
0,U103,2018-04-20 09:30:00,2020-08-12 22:30:00
1,U125,2017-07-13 14:45:00,2021-10-25 15:00:00
2,U137,2017-06-19 02:15:00,2022-08-08 23:45:00
3,U14,2020-02-29 00:15:00,2022-08-06 23:45:00
4,U19,2017-08-29 13:15:00,2021-12-31 23:45:00
5,U228,2017-03-18 00:15:00,2021-10-25 15:00:00
6,U266,2020-03-03 10:00:00,2021-12-31 23:45:00
7,U268,2020-03-03 10:00:00,2022-08-06 23:45:00
8,U354,2020-02-29 00:15:00,2022-08-06 23:45:00
9,U376,2020-05-12 17:00:00,2021-12-31 23:45:00


In [56]:
# Calculate common time range across all users
common_min_time = time_range_df['Min Time'].max()
common_max_time = time_range_df['Max Time'].min()

# Add common time columns to your existing DataFrame
time_range_df['Common Min Time'] = common_min_time
time_range_df['Common Max Time'] = common_max_time

# Display the result
time_range_df


,User,Min Time,Max Time,Common Min Time,Common Max Time
0,U103,2018-04-20 09:30:00,2020-08-12 22:30:00,2020-05-12 17:00:00,2020-08-12 22:30:00
1,U125,2017-07-13 14:45:00,2021-10-25 15:00:00,2020-05-12 17:00:00,2020-08-12 22:30:00
2,U137,2017-06-19 02:15:00,2022-08-08 23:45:00,2020-05-12 17:00:00,2020-08-12 22:30:00
3,U14,2020-02-29 00:15:00,2022-08-06 23:45:00,2020-05-12 17:00:00,2020-08-12 22:30:00
4,U19,2017-08-29 13:15:00,2021-12-31 23:45:00,2020-05-12 17:00:00,2020-08-12 22:30:00
5,U228,2017-03-18 00:15:00,2021-10-25 15:00:00,2020-05-12 17:00:00,2020-08-12 22:30:00
6,U266,2020-03-03 10:00:00,2021-12-31 23:45:00,2020-05-12 17:00:00,2020-08-12 22:30:00
7,U268,2020-03-03 10:00:00,2022-08-06 23:45:00,2020-05-12 17:00:00,2020-08-12 22:30:00
8,U354,2020-02-29 00:15:00,2022-08-06 23:45:00,2020-05-12 17:00:00,2020-08-12 22:30:00
9,U376,2020-05-12 17:00:00,2021-12-31 23:45:00,2020-05-12 17:00:00,2020-08-12 22:30:00


In [62]:
import os
import pandas as pd

# Assuming combined_df is already created from pd.concat(dataframes, ignore_index=True)
directory = r'C:/Users/dmarc/Desktop/EWELD_Project/EWELD/'
file_path = os.path.join(directory, 'ec_CT1.csv')

# Create the directory if it does not exist
os.makedirs(directory, exist_ok=True)

# Save the DataFrame to a CSV file (without the index column)
pivot_df_ec_CT1.to_csv(file_path, index=False)

In [ ]:
pivot_df_ec_CT1_noU103 = pivot_df_ec_CT1.drop('U103', axis=1).copy()

In [58]:
# Get user columns (exclude Time and City)
user_columns = pivot_df_ec_CT1_noU103.columns.difference(['Time', 'City'])

# Initialize dictionaries to store results
min_times = {}
max_times = {}

for user in user_columns:
    # Filter rows where the user's value is not NaN
    valid_rows = pivot_df_ec_CT1_noU103[pivot_df_ec_CT1_noU103[user].notna()]
    
    if not valid_rows.empty:
        min_times[user] = valid_rows['Time'].min()
        max_times[user] = valid_rows['Time'].max()
    else:
        min_times[user] = pd.NaT  # Handle users with no data
        max_times[user] = pd.NaT

# Create the summary DataFrame
time_range_df = pd.DataFrame({
    'User': list(min_times.keys()),
    'Min Time': list(min_times.values()),
    'Max Time': list(max_times.values())
})
# Calculate common time range across all users
common_min_time = time_range_df['Min Time'].max()
common_max_time = time_range_df['Max Time'].min()

# Add common time columns to your existing DataFrame
time_range_df['Common Min Time'] = common_min_time
time_range_df['Common Max Time'] = common_max_time

# Display the result
time_range_df

,User,Min Time,Max Time,Common Min Time,Common Max Time
0,U125,2017-07-13 14:45:00,2021-10-25 15:00:00,2020-05-12 17:00:00,2021-10-25 15:00:00
1,U137,2017-06-19 02:15:00,2022-08-08 23:45:00,2020-05-12 17:00:00,2021-10-25 15:00:00
2,U14,2020-02-29 00:15:00,2022-08-06 23:45:00,2020-05-12 17:00:00,2021-10-25 15:00:00
3,U19,2017-08-29 13:15:00,2021-12-31 23:45:00,2020-05-12 17:00:00,2021-10-25 15:00:00
4,U228,2017-03-18 00:15:00,2021-10-25 15:00:00,2020-05-12 17:00:00,2021-10-25 15:00:00
5,U266,2020-03-03 10:00:00,2021-12-31 23:45:00,2020-05-12 17:00:00,2021-10-25 15:00:00
6,U268,2020-03-03 10:00:00,2022-08-06 23:45:00,2020-05-12 17:00:00,2021-10-25 15:00:00
7,U354,2020-02-29 00:15:00,2022-08-06 23:45:00,2020-05-12 17:00:00,2021-10-25 15:00:00
8,U376,2020-05-12 17:00:00,2021-12-31 23:45:00,2020-05-12 17:00:00,2021-10-25 15:00:00
9,U382,2020-02-29 00:15:00,2021-12-31 23:45:00,2020-05-12 17:00:00,2021-10-25 15:00:00


In [59]:
pivot_df_ec_CT1_noU103.head()

User,Time,City,U125,U137,U14,U19,U228,U266,U268,U354,U376,U382,U50
0,2017-03-18 00:15:00,CT1,NaN,NaN,NaN,NaN,404.0065,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-03-18 00:30:00,CT1,NaN,NaN,NaN,NaN,402.3807,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-03-18 00:45:00,CT1,NaN,NaN,NaN,NaN,413.3547,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-03-18 01:00:00,CT1,NaN,NaN,NaN,NaN,396.6905,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-03-18 01:15:00,CT1,NaN,NaN,NaN,NaN,407.6645,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
shape = pivot_df_ec_CT1_noU103.shape
print("Data Frane pivot_df_ec_CT1 shape = {}".format(shape))

print('----\n')
missing_values = pivot_df_ec_CT1_noU103.drop(columns=['Time', 'City']).isna().sum()
missing_values_dict = missing_values.to_frame
print("missing_values by User {}".format(missing_values_dict))

Data Frane pivot_df_ec_CT1 shape = (189119, 13)
----

missing_values by User <bound method Series.to_frame of User
U125     38877
U137      8936
U14     103680
U19      36916
U228     27587
U266    124935
U268    104007
U354    103680
U376    131683
U382    124608
U50      58752
dtype: int64>


In [61]:
import os
import pandas as pd

# Assuming combined_df is already created from pd.concat(dataframes, ignore_index=True)
directory = r'C:/Users/dmarc/Desktop/EWELD_Project/EWELD/'
file_path = os.path.join(directory, 'ec_CT1_noU103.csv')

# Create the directory if it does not exist
os.makedirs(directory, exist_ok=True)

# Save the DataFrame to a CSV file (without the index column)
pivot_df_ec_CT1_noU103.to_csv(file_path, index=False)